# 1.2 OpenAI API - Advanced Features

This notebook covers **advanced OpenAI API features** for building sophisticated AI agents:

1. **Function Calling & Tool Role**: Enable AI to call external tools
2. **Reasoning Modes**: Extended thinking for complex problems
3. **Model Selection**: Choose the right GPT-5 model for your use case
4. **Advanced Role Patterns**: Developer vs user role hierarchy

**Prerequisites:** Complete notebook 1.1 (OpenAI API Basics) first.

<a target="_blank" href="https://colab.research.google.com/github/IT-HUSET/ai-agents-course-2025/blob/main/exercises/1.2-openai-api-advanced.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup

In [ ]:
%pip install openai~=1.60 python-dotenv~=1.0 --upgrade --quiet

In [ ]:
import os
import json
import time
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

_ = load_dotenv(find_dotenv())
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("✅ OpenAI client initialized")

---

## Part 1: Function Calling & Tool Role

Function calling enables AI models to interact with external tools and APIs.

### The Tool Role - For Function Calling

The **`tool`** role is used when the model calls a function and you need to provide the result back.

**Function Calling Flow:**
```
1. User asks a question
2. Model decides to call a tool → returns tool_call
3. You execute the function
4. You send result back with role: "tool" ← This is the tool role
5. Model uses result to answer user
```

**Chat Completions API Roles:**

| Role | Purpose | Who Creates It |
|------|---------|----------------|
| `system` | Persistent behavior instructions | Developer |
| `user` | User's queries | User/Developer |
| `assistant` | Model's responses | Model |
| `tool` | Function call results | Developer |

**Important:** You'll see the `tool` role in action in the example below.

In [ ]:
# Example: Complete function calling flow with tool role
import json

# Define a simple function
def get_weather(city: str) -> dict:
    """Fake weather API"""
    return {
        "city": city,
        "temperature": 22,
        "condition": "sunny",
        "humidity": 65
    }

# Define tool for the model
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current weather for a city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city name"
                }
            },
            "required": ["city"]
        }
    }
}

# Step 1: User asks a question
messages = [
    {"role": "user", "content": "What's the weather in Stockholm?"}
]

print("Step 1: User asks question")
print(f"Messages: {len(messages)}")
print()

# Step 2: Model decides to call the tool
response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages,
    tools=[weather_tool],
    tool_choice="auto"
)

message = response.choices[0].message
print("Step 2: Model wants to call tool")
print(f"Tool calls: {message.tool_calls}")
print()

# Add assistant's tool call to history
messages.append(message)

# Step 3 & 4: Execute function and add result with 'tool' role
if message.tool_calls:
    tool_call = message.tool_calls[0]
    function_args = json.loads(tool_call.function.arguments)
    
    # Execute the function
    result = get_weather(**function_args)
    
    print("Step 3: Execute function")
    print(f"Function: {tool_call.function.name}")
    print(f"Arguments: {function_args}")
    print(f"Result: {result}")
    print()
    
    # Add function result with 'tool' role
    messages.append({
        "role": "tool",  # ← THE TOOL ROLE!
        "tool_call_id": tool_call.id,
        "content": json.dumps(result)
    })
    
    print("Step 4: Add result with 'tool' role")
    print(f"Messages now: {len(messages)}")
    print()

# Step 5: Model generates final answer using the tool result
final_response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages
)

print("Step 5: Model's final answer:")
print(final_response.choices[0].message.content)
print()

print("\n" + "="*80)
print("Final message history:")
for i, msg in enumerate(messages):
    role = msg.get('role') if isinstance(msg, dict) else msg.role
    print(f"{i+1}. {role}")

### Tool Preambles (GPT-5 Feature)

**Preambles** are brief explanations GPT-5 generates *before* calling a tool, explaining its intent.

**Benefits:**
- 🔍 **Transparency**: See why the model chose a tool
- 🐛 **Debuggability**: Easier to understand failures
- ✅ **Accuracy**: Improves tool-calling success rate
- 🎯 **User experience**: Users see the reasoning

**How to enable:** Add an instruction asking the model to explain before calling tools.

In [ ]:
# Example: Tool calling WITH preambles
messages_with_preamble = [
    {
        "role": "system",
        "content": "Before calling any tool, briefly explain why you're calling it."
    },
    {
        "role": "user",
        "content": "What's the weather in Stockholm and Paris? Compare them."
    }
]

print("Tool calling WITH preambles:")
print("="*80)

response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages_with_preamble,
    tools=[weather_tool],
    tool_choice="auto"
)

message = response.choices[0].message

# The model will explain its reasoning before calling tools
if message.content:
    print("\n🧠 Preamble (model's explanation):")
    print(message.content)

if message.tool_calls:
    print(f"\n🛠️  Tool calls: {len(message.tool_calls)}")
    for tool_call in message.tool_calls:
        print(f"  - {tool_call.function.name}({tool_call.function.arguments})")

**Preamble Example Output:**

```
🧠 Preamble (model's explanation):
I'll check the weather for both Stockholm and Paris so I can compare them.

🛠️ Tool calls: 2
  - get_weather({"city": "Stockholm"})
  - get_weather({"city": "Paris"})
```

This makes the model's reasoning visible and improves user trust!

### 🎯 Exercise 1: Build a Multi-Tool Agent

**Task:** Create an agent with multiple tools:
1. `get_weather(city)` - get weather
2. `get_time(city)` - get current time
3. `convert_currency(amount, from_currency, to_currency)` - currency conversion

Test it with: "What's the weather in Tokyo, what time is it there, and how much is 100 USD in JPY?"

In [ ]:
# YOUR CODE HERE

# Define your functions
def get_time(city: str) -> dict:
    # TODO: Implement (can be fake data for this exercise)
    pass

def convert_currency(amount: float, from_currency: str, to_currency: str) -> dict:
    # TODO: Implement (can be fake data for this exercise)
    pass

# Define your tools
# TODO: Create tool definitions for all three functions

# Test your multi-tool agent
# TODO: Implement the agent loop

---

## Part 2: Reasoning Modes

GPT-5 models support **reasoning modes** - extended thinking for complex problems.

### Without Reasoning Mode

In [ ]:
import time

complex_problem = """A farmer has chickens and rabbits. 
There are 35 heads and 94 legs total. 
How many chickens and how many rabbits are there?
"""

# Regular mode
start = time.time()
response_normal = client.responses.create(
    model="gpt-5-mini",
    input=complex_problem
)
time_normal = time.time() - start

print("WITHOUT Reasoning Mode:")
print(f"Time: {time_normal:.2f}s")
print(response_normal.output_text)

### With Reasoning Mode

In [ ]:
# GPT-5 with high reasoning effort
start = time.time()
response_reasoning = client.chat.completions.create(
    model="gpt-5",
    reasoning_effort="high",  # GPT-5-mini and GPT-5 support reasoning
    messages=[
        {"role": "user", "content": complex_problem}
    ]
)
time_reasoning = time.time() - start

print("WITH Reasoning Mode (GPT-5 with high reasoning):")
print(f"Time: {time_reasoning:.2f}s")

# Check if reasoning tokens are available
if hasattr(response_reasoning.choices[0].message, 'reasoning'):
    print("\n🧠 Internal Reasoning (preview):")
    reasoning = response_reasoning.choices[0].message.reasoning or "[Hidden]"
    print(reasoning[:500] + "..." if len(reasoning) > 500 else reasoning)
    print("\n" + "="*80 + "\n")

print("Final Answer:")
print(response_reasoning.choices[0].message.content)

### Inspecting Reasoning Tokens

In [ ]:
# Examine token usage
print("Token Usage Analysis:")
print(f"Prompt tokens: {response_reasoning.usage.prompt_tokens}")
print(f"Completion tokens: {response_reasoning.usage.completion_tokens}")

if hasattr(response_reasoning.usage, 'reasoning_tokens'):
    # GPT-5 includes reasoning tokens when using reasoning_effort
    print(f"Reasoning tokens: {response_reasoning.usage.reasoning_tokens}")
    print(f"\n💡 Model spent {response_reasoning.usage.reasoning_tokens} tokens 'thinking' internally")

print(f"Total tokens: {response_reasoning.usage.total_tokens}")

### Comparing Reasoning Quality

In [ ]:
# Complex coding problem
coding_problem = """Find all bugs in this Python code and explain how to fix them:

def calculate_average(numbers):
    total = 0
    for i in range(len(numbers)):
        total = total + numbers[i]
    return total / len(numbers)

result = calculate_average([1, 2, 3, 4, 5])
print(f"Average: {result}")
"""

print("GPT-5-mini (fast):")
response_fast = client.responses.create(
    model="gpt-5-mini",
    input=coding_problem
)
print(response_fast.output_text)
print("\n" + "="*80 + "\n")

print("GPT-5 with high reasoning (reasoning):")
response_smart = client.chat.completions.create(
    model="gpt-5",
    reasoning_effort="high",
    messages=[{"role": "user", "content": coding_problem}]
)
print(response_smart.choices[0].message.content)

### 🎯 Exercise 2: Test Reasoning Capabilities

**Task:** Compare GPT-5-mini vs GPT-5 on these problems:
1. A logic puzzle of your choice
2. A multi-step math problem
3. A code optimization challenge

**Analyze:**
- Which model gives more accurate answers?
- How much longer does reasoning take?
- When is the extra cost worth it?

In [ ]:
# YOUR CODE HERE

logic_puzzle = """TODO: Write a challenging logic puzzle"""

def compare_models(problem: str):
    """Compare GPT-5-mini vs GPT-5 with high reasoning on a problem"""
    # TODO: Implement comparison
    pass

# Test your puzzles
# compare_models(logic_puzzle)

---

## Part 3: Model Selection & Pricing

**GPT-5** (released August 2024) is the latest model family:

### Complete Model Selection Guide

| Feature | GPT-4o | GPT-5 | GPT-5-mini | GPT-5-nano |
|---------|--------|-------|------------|------------|
| **Context Window** | 128K | 400K | 400K | 400K |
| **Max Output** | 16,384 | 128K | 128K | 128K |
| **Knowledge Cutoff** | Oct 2023 | Sep 2024 | May 2024 | May 2024 |
| **Reasoning Levels** | ❌ None | 🤔🤔🤔🤔 (4/4) | 🤔🤔🤔 (3/4) | 🤔🤔 (2/4) |
| **Speed** | ⚡⚡⚡ | ⚡⚡⚡ | ⚡⚡⚡⚡ | ⚡⚡⚡⚡⚡ |
| **Input ($/1M)** | $2.50 | $1.25 | $0.25 | $0.05 |
| **Cached Input** | $1.25 | $0.13 | $0.03 | $0.01 |
| **Output ($/1M)** | $10.00 | $10.00 | $2.00 | $0.40 |
| **Vision** | ✅ | ✅ | ✅ | ✅ |
| **Audio** | ✅ | ✅ | ✅ | ✅ |
| **Reasoning Tokens** | ❌ | ✅ | ✅ | ✅ |
| **Best For** | Legacy | Best quality | Balanced | Simple/cheap |

**Key Insights:**
- **GPT-5 is cheaper than GPT-4o** for input ($1.25 vs $2.50)
- **All GPT-5 models** have 400K context (3x larger than GPT-4o)
- **Reasoning tokens** are included when using reasoning_effort
- **Cached inputs** are ~90% cheaper (great for repeated prompts)

**Reasoning Levels Explained:**
- **GPT-5**: minimal, low, medium, high (all 4)
- **GPT-5-mini**: minimal, low, medium (no high)
- **GPT-5-nano**: minimal, low (basic reasoning only)

**Decision Tree:**
```
What's your use case?

Complex reasoning (math, code debugging, strategy)?
├─ Need highest quality? → GPT-5 (reasoning_effort="high")
└─ Budget conscious? → GPT-5-mini (reasoning_effort="medium")

Standard tasks (chat, content, analysis)?
├─ Need best quality? → GPT-5
├─ Balanced quality/cost? → GPT-5-mini
└─ Simple/repetitive? → GPT-5-nano

Cost is primary concern?
├─ Simple tasks → GPT-5-nano ($0.45/1M total)
└─ Complex tasks → GPT-5-mini ($2.25/1M total)
```

**Cost Comparison (1M input + 1M output):**
```python
# Without caching:
GPT-5-nano:  $0.05 + $0.40  = $0.45   (10x cheaper than GPT-4o)
GPT-5-mini:  $0.25 + $2.00  = $2.25   (4x cheaper than GPT-4o)
GPT-5:       $1.25 + $10.00 = $11.25  (same output cost as GPT-4o)
GPT-4o:      $2.50 + $10.00 = $12.50  (legacy)

# With 90% cached input (common in agents):
GPT-5-nano:  $0.01 + $0.40  = $0.41   (97% cheaper!)
GPT-5-mini:  $0.03 + $2.00  = $2.03
GPT-5:       $0.13 + $10.00 = $10.13
```

**When to use reasoning_effort:**
- **Don't specify** (defaults to auto) - for most tasks
- **"low"** - Light reasoning, faster
- **"medium"** - Balanced (good default for complex tasks)
- **"high"** - Maximum reasoning (GPT-5 only, for hardest problems)

**⚠️ Note:** Higher reasoning effort increases reasoning tokens (hidden cost added to output tokens)

---

## Part 4: Advanced Role Patterns

Understanding the role hierarchy in different APIs.

### Developer and User Roles in Responses API

The Responses API introduces a **role hierarchy** different from Chat Completions:

**Roles in Responses API:**
- **`developer`**: Instructions from the application developer (highest priority)
- **`user`**: Instructions/input from the end user (lower priority)
- **`assistant`**: Model's responses

**Priority:** `developer` > `user`

The `instructions` parameter is shorthand for a `developer` role message. These are equivalent:

In [ ]:
# Method 1: Using instructions parameter
response1 = client.responses.create(
    model="gpt-5-mini",
    instructions="Talk like a pirate.",
    input="Are semicolons optional in JavaScript?"
)

print("Method 1 - Using instructions parameter:")
print(response1.output_text)
print("\n" + "="*80 + "\n")

# Method 2: Using developer role in input array (equivalent)
response2 = client.responses.create(
    model="gpt-5-mini",
    input=[
        {
            "role": "developer",
            "content": "Talk like a pirate."
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

print("Method 2 - Using developer role in input:")
print(response2.output_text)

print("\n💡 Both methods produce similar results. Use `instructions` for simple cases, `developer` role for more complex message structures.")

### Understanding Role Priority

**In Chat Completions API:**
- `system` role provides persistent instructions
- All roles are treated equally in conversation flow

**In Responses API:**
- `developer` role has **higher priority** than `user` role
- This ensures app-level instructions override user attempts to change behavior
- Critical for building safe, reliable applications

---

## Summary

In this notebook, you learned:

✅ **Function Calling**: How to use the `tool` role to integrate external functions  
✅ **Tool Preambles**: GPT-5 feature for transparent reasoning  
✅ **Reasoning Modes**: Extended thinking with `reasoning_effort` parameter  
✅ **Model Selection**: Complete guide to GPT-5, GPT-5-mini, and GPT-5-nano  
✅ **Role Hierarchy**: Developer vs user roles in Responses API

**Key Takeaways:**
- Function calling enables AI to interact with external tools
- Reasoning modes improve quality on complex problems (at higher cost)
- Choose the right GPT-5 model based on your use case and budget
- Understand role priorities when building production applications

**Next Steps:**
- **Notebook 1.3**: Structured outputs with JSON schemas
- **Notebook 1.4**: Prompt engineering techniques
- **Notebook 1.5**: Context management strategies
- **Notebook 1.6**: Agentic applications

**Resources:**
- [Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [Reasoning Models Guide](https://platform.openai.com/docs/guides/reasoning)
- [Model Pricing](https://openai.com/pricing)